In [31]:

!pip install spacy transformers scikit-learn stanza


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for stanza from https://files.pythonhosted.org/packages/3f/8f/7ad1149d082799c7bed0a8ecc7e8bcee11aa47d122c8b793f580a1178e75/stanza-1.8.2-py3-none-any.whl.metadata
  Obtaining dependency information for emoji from https://files.pythonhosted.org/packages/e6/90/20ad30babfa8f2b5ab46281d8e17bdfdbb3ac294cda14d525b9c2d958846/emoji-2.12.1-py3-none-any.whl.metadata
  Obtaining dependency information for toml from https://files.pythonhosted.org/packages/44/6f/7120676b6d73228c96e17f1f794d8ab046fc910d781c8d151120c3f1569e/toml-0.10.2-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.1/990.1 kB 9.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 20.2 MB/s eta 0:00:00
Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [33]:

import stanza
from transformers import AutoModelForMaskedLM, AutoTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


In [34]:


stanza.download('en')
nlp = stanza.Pipeline('en', processors='tokenize,ner')

model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name)



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


2024-08-04 12:32:25 INFO: Downloaded file to /Users/asharma/stanza_resources/resources.json
2024-08-04 12:32:25 INFO: Downloading default packages for language: en (English) ...


2024-08-04 12:32:51 INFO: Downloaded file to /Users/asharma/stanza_resources/en/default.zip
2024-08-04 12:32:53 INFO: Finished downloading models and saved to /Users/asharma/stanza_resources
2024-08-04 12:32:53 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-08-04 12:32:53 INFO: Downloaded file to /Users/asharma/stanza_resources/resources.json
2024-08-04 12:32:53 WARNING: Language en package default expects mwt, which has been added
2024-08-04 12:32:53 INFO: Loading these models for language: en (English):
| Processor | Package                   |
-----------------------------------------
| tokenize  | combined                  |
| mwt       | combined                  |
| ner       | ontonotes-ww-multi_charlm |

2024-08-04 12:32:53 INFO: Using device: cpu
2024-08-04 12:32:53 INFO: Loading: tokenize
2024-08-04 12:32:53 INFO: Loading: mwt
2024-08-04 12:32:53 INFO: Loading: ner
2024-08-04 12:32:54 INFO: Done loading processors!
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model tr

In [39]:

# Data Preparation
def prepare_data(context, mask_token):
    masked_context = context.replace(mask_token, tokenizer.mask_token)
    token_ids = tokenizer.encode(masked_context, return_tensors='pt')
    return token_ids

# Generate Candidates
def generate_candidates(token_ids):
    masked_index = torch.where(token_ids == tokenizer.mask_token_id)[1]
    with torch.no_grad():
        outputs = model(token_ids)
    predictions = outputs.logits[0, masked_index, :].squeeze(0)
    return predictions

# Contextual Matching
def contextual_matching(predictions, top_k=5):
    top_k_tokens = torch.topk(predictions, top_k, dim=0).indices.tolist()
    candidates = [tokenizer.decode([token]) for token in top_k_tokens]
    return candidates

# Validation
def validate_replacement(original_context, obfuscated_context):
    original_ids = tokenizer.encode(original_context, return_tensors='pt')
    obfuscated_ids = tokenizer.encode(obfuscated_context, return_tensors='pt')

    with torch.no_grad():
        original_hidden_states = bert_model(original_ids)[0].mean(dim=1).numpy()
        obfuscated_hidden_states = bert_model(obfuscated_ids)[0].mean(dim=1).numpy()
    
    similarity = cosine_similarity(original_hidden_states, obfuscated_hidden_states)
    return similarity[0][0]

# Identify Mask Candidates
def identify_mask_candidates(context, entity_types):
    doc = nlp(context)
    mask_candidates = [ent.text for ent in doc.ents if ent.type in entity_types]
    return mask_candidates

In [40]:
# Main function
def main(context, entity_types):
    mask_candidates = identify_mask_candidates(context, entity_types)
    results = {}
    for candidate in mask_candidates:
        masked_context = context.replace(candidate, '[MASK]')
        token_ids = prepare_data(masked_context, '[MASK]')
        predictions = generate_candidates(token_ids)
        candidates = contextual_matching(predictions)
        best_candidate = max(candidates, key=lambda c: validate_replacement(context, context.replace(candidate, c)))
        results[candidate] = best_candidate
    return results

In [41]:
context = """Apple Intelligence enables delightful new ways to express yourself visually. 
Create fun, original images and brand-new Genmoji that are truly personal to you. 
Turn a rough sketch into a related image that complements your notes with Image Wand. 
And make a custom memory movie based on the description you provide."""


In [47]:
context = """
With ChatGPT from OpenAI integrated into Siri and Writing Tools, 
you get even more expertise when it might be helpful for you no need to jump between tools. 
Siri can tap into ChatGPT for certain requests, including questions about photos or documents. 
And with Compose in Writing Tools, you can create and illustrate original content from scratch.
You control when ChatGPT is used and will be asked before any of your information is shared. 
Anyone can access ChatGPT for free, without creating an account. 
ChatGPT subscribers can connect accounts to access paid features within these experiences.
"""

In [48]:

entity_types = ['GPE', 'ORG'] 

#context = "An event occurred in New York near the Empire State Building."
candidates = main(context, entity_types)
print("Candidates for replacement:", candidates)

TypeError: argument 'ids': 'list' object cannot be interpreted as an integer

In [49]:
#context = "An event occurred in New York near the Empire State Building on January 1st, 2024."
entity_types = ['PERSON', 'NORP', 'FAC', 'ORG', 'GPE', 'LOC', 'PRODUCT', 'EVENT', 'WORK_OF_ART', 'LAW', 
                'LANGUAGE', 'DATE', 'TIME', 'PERCENT', 'MONEY', 'QUANTITY']
candidates = main(context, entity_types)
print("Candidates for replacement:", candidates)

TypeError: argument 'ids': 'list' object cannot be interpreted as an integer